Connect to Snowflake w/ the Python Connector and SNOWSQL config file data

In [ ]:
# see https://github.com/Snowflake-Labs/sfguide-snowpark-scikit-learn/blob/main/3_snowpark_end_to_end_ml.ipynb

from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
session = Session.builder.configs(SnowflakeLoginOptions("test_conn")).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

Get 10% sample data from the HOUSING table, and separate MEDIAN_HOUSE_VALUE

In [ ]:
df = session.table("HOUSING").sample(frac=0.10).to_pandas()
df.shape

X = df.loc[:, df.columns != 'MEDIAN_HOUSE_VALUE']
X

Perform data preprocessing in a transformation pipeline

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

COL_NAMES = ['OCEAN_PROXIMITY',
    'LONGITUDE', 'LATITUDE', 'HOUSING_MEDIAN_AGE', 'TOTAL_ROOMS',
    'TOTAL_BEDROOMS', 'POPULATION', 'HOUSEHOLDS', 'MEDIAN_INCOME']
pipe = ColumnTransformer([
    ( "num", Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler())]), COL_NAMES[1:]),
    ( "cat", Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))]), COL_NAMES[0:0])
])
model = pipe.fit_transform(X)
model

Check changes on numeric features

In [ ]:
import pandas as pd
df_out = pd.DataFrame(model, index=df.index, columns=COL_NAMES[1:])
df_out.head(10)